# Fairness with Fairlearn

## Defining Fairness

To address fairness you have to define it first, in this case we use define it as harm to a person. I really enjoy the types of harm that Kate Crawford introduced, so lets focus on harm of allocation. This is defined as AI systems withholding or extending opportunities, resources or information. For example with our tumor data, we underpredict tumors for females than males, therefore we don't enroll them in a treatment plan. This can also be a quality-of-service harm meaning we don't provide the same level of service across groups.

## Quantify Fairness

Now that we have defined it, how can we measure it. This is dependent on our model objects, in this case we have predictions, features, and our sensitive features (protected class). An ideal model would have similar outcomes for all groups regardless of race, this would mean race is independent of our response, this is defined as demographic parity.

However the issue being addressed is a disparity, so we define disparity as a difference in terms of differences in our ability to predict tumors across race. The primary metric we will use is recall, because we do not want to diagnose a tumor as benign when in reality it was harmful. We want to minimize false negatives in this case and want this minimization across race.

## Fairlearn: Assessments

Fairlearn is broken out into two main components. The first component is the assessment, this helps us evaluate demographic parity differences. The second component helps in addressing the disparity. 

In [61]:
import pickle
import pandas as pd
import numpy as np

pickle_dir = "../data/prepped_data.dat"

with open(pickle_dir, "rb") as f:
    data = pickle.load(f)

X_train_prepped, y_train, X_val, y_val, X_test, y_test, preprocessing = data    

X_test_trans = preprocessing.fit_transform(X_test)

X_test_prepped = pd.DataFrame(
    X_test_trans, columns=preprocessing.get_feature_names_out())

X_test_prepped

,num__Age_at_diagnosis,cat__Race_Other,cat__Race_black or african american,cat__Race_white,binary__Gender,binary__IDH1,binary__TP53,binary__ATRX,binary__PTEN,binary__EGFR,...,binary__FUBP1,binary__RB1,binary__NOTCH1,binary__BCOR,binary__CSMD3,binary__SMARCA4,binary__GRIN2A,binary__IDH2,binary__FAT4,binary__PDGFRA
0,0.573547,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.517942,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.775059,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.643980,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.309164,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0.383600,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,0.379893,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0.512011,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0.357651,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [31]:
cat_pipeline = preprocessing.transformers[1]

ohe_transformer  = cat_pipeline[1]

ohe_race = ohe_transformer.inverse_transform(ohe_transformer.fit_transform(X_test))[:, 2]

ohe_race

array(['white', 'white', 'white', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'white', 'white', 'white',
       'black or african american', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'white', 'white', 'white', 'white',
       'Other', 'white', 'white', 'black or african american', 'white',
       'white', 'white', 'black or african american', 'white', 'white',
       'white', 'white', 'white', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'white', 'white', 'white',
       'black or african american', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'white', 'white', 'white', 'white',
       'white', 'white', 'white', 'black or african american', 'white',
       'white', 'white', 'black or african american', 'white', 'Other',
       'white', 'white', 'white', 'white', 'white', 'white', 'white'],
      dtype=object)

In [43]:
import tensorflow as tf

reconstructed_model = tf.keras.models.load_model("../model/my_model.keras")

y_proba = reconstructed_model.predict(X_test_prepped)

# Convert probabilities to binary values
threshold = 0.5
binary_pred = (y_proba >= threshold).astype(int)

#reconstructed_model.evaluate(X_test_prepped, y_test)

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/stepWARNING:tensorflow:6 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001BDFFF4B640> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


array([[0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
    

In [38]:
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import count, false_positive_rate, selection_rate
from sklearn.metrics import recall_score

my_metrics = {
    'recall' : recall_score,
    'count' : count
}

mf = MetricFrame(
    metrics = my_metrics,
    y_true = y_test,
    y_pred = binary_pred, 
    sensitive_features = ohe_race
)


mf.overall

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


recall     0.891892
count     84.000000
dtype: float64

In [39]:
mf.by_group

,recall,count
sensitive_feature_0,,
Other,0.00000,2.0
black or african american,0.80000,6.0
white,0.90625,76.0


In [91]:
pd.DataFrame({'difference': mf.difference(),
              'group_min': mf.group_min(),
              'group_max': mf.group_max()}).T


,recall,count
difference,0.90625,74.0
ratio,0.0,0.026316
group_min,0.0,2.0
group_max,0.90625,76.0


In [42]:
from fairlearn.metrics import demographic_parity_difference

demographic_parity_difference(y_test,
                              binary_pred,
                              sensitive_features = ohe_race )

0.8333333333333334

To get an idea of how fair our model is we quantify this with the demographic parity difference, or difference between the largest and smallest group-level selection rate. 

Selection rate in Fairlearn is the percentage of data points in each class classified as the positive label. (e.g. 25/50 for men and 2/10 for women) 

number positive predictions / total number of data points for the positive class

In [63]:
results_df = pd.DataFrame({'race': ohe_race, 'pred': binary_pred_flatten, 'actual': y_test_flatten})

results_df

,race,pred,actual
0,white,0,0
1,white,1,1
2,white,1,0
3,white,0,0
4,white,0,1
...,...,...,...
79,white,0,0
80,white,0,0
81,white,1,1
82,white,0,0


In [59]:
#binary_pred.shape

binary_pred_flatten = binary_pred.flatten()

binary_pred_flatten.shape

(84,)

In [57]:
ohe_race.shape

(84,)

In [62]:
y_test_flatten = y_test.to_numpy().flatten()

y_test_flatten.shape

(84,)

A few terms used by fair learn to understand use of the demographic_parity_difference function.

Selection rate is defined as the fraction of predicted labels matching the good outcome. The demographic parity difference takes the difference between largest and smallest group-level selection rate. 


In [90]:
#from fairlearn import selection_rate

#selection_rate(y_true, y_pred,  pos_label=1)

new_df = results_df[results_df['race'] != 'Other']


group_selection_rates = new_df.groupby('race').apply(lambda group: group['pred'].sum() / len(group['actual']) )

group_selection_rates


C:\Users\marti\AppData\Local\Temp\ipykernel_4816\861305877.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_selection_rates = new_df.groupby('race').apply(lambda group: group['pred'].sum() / len(group['actual']) )


race
black or african american    0.833333
white                        0.500000
dtype: float64

In [71]:
from fairlearn.metrics import selection_rate

group_selection_rates = new_df.groupby('race').apply(lambda group: selection_rate(y_true=group['actual'],y_pred= group['pred'],pos_label=1))
group_selection_rates

C:\Users\marti\AppData\Local\Temp\ipykernel_4816\2906087685.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_selection_rates = new_df.groupby('race').apply(lambda group: selection_rate(y_true=group['actual'],y_pred= group['pred'],pos_label=1))


race
black or african american    0.833333
white                        0.500000
dtype: float64

In [74]:
s_w = np.ones(len(binary_pred_flatten))

s_w.sum()

84.0

In [81]:
s_w.shape

(84,)

In [82]:
subset_array = binary_pred[binary_pred == 1]

subset_array.shape

(43,)

In [89]:
subset_arrayT = np.conj(subset_array).T

In [88]:
np.dot(subset_arrayT, s_w)

ValueError: shapes (43,) and (84,) not aligned: 43 (dim 0) != 84 (dim 0)

In [ ]:
np.dot(selected, s_w) / s_w.sum()

simple example

In [92]:
y_true = [1,1,1,1,1,0,0,1,1,0]
y_pred = [0,1,1,1,1,0,0,0,1,1]
sex = ['Female']*5 + ['Male']*5

In [108]:
from fairlearn.metrics import MetricFrame, selection_rate
import pandas as pd


test = pd.DataFrame({'sex': sex, 'pred': y_pred, 'actual': y_true})

test

,sex,pred,actual
0,Female,0,1
1,Female,1,1
2,Female,1,1
3,Female,1,1
4,Female,1,1
5,Male,0,0
6,Male,0,0
7,Male,0,1
8,Male,1,1
9,Male,1,0


In [109]:
group_selection_rates = test.groupby('sex').apply(lambda group: group['pred'].sum() / len(group['actual']) )

group_selection_rates

C:\Users\marti\AppData\Local\Temp\ipykernel_4816\1783534178.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_selection_rates = test.groupby('sex').apply(lambda group: group['pred'].sum() / len(group['actual']) )


sex
Female    0.8
Male      0.4
dtype: float64

In [114]:
metrics = {"selection_rate": selection_rate}
mf1 = MetricFrame(
     metrics=metrics,
     y_true=test['actual'],
     y_pred=test['pred'],
     sensitive_features=test['sex'])

In [115]:
mf1.by_group 

,selection_rate
sex,
Female,0.8
Male,0.4


In [116]:
mf1.difference(method='between_groups')

selection_rate    0.4
dtype: float64

In [119]:
from fairlearn.metrics import demographic_parity_difference

print(demographic_parity_difference( y_true=test['actual'],
                                y_pred=test['pred'],
                             sensitive_features=test['sex'],
                               method='between_groups'))

0.4


# Fairlearn: Mitigations

The second component two Fairlearn is addresses the actual differences in our metrics. This can be done before training or following training. From my experience, its faster to do this in the later stages following training, doing this in the preprocessing stage involves lot of reiterations and is not as fast.

In [ ]:
from fairlearn.postprocessing import ThresholdOptimizer

threshold_optimizer = ThresholdOptimizer(
    estimator=pipeline,
    constraints="true_positive_rate_parity",
    objective="balanced_accuracy_score",
    predict_method="predict_proba",
    prefit=False,
)